# RESTAURANT DISH RECOMMENDER SYSTEM

#### I poped into one of the restaurant's today & was confused after seeing the long menu card and innovative names. 
#### Sometimes a common man might not know what the dish actually means so he just needs to know what do I order from this long menu.
#### In such cases earlier reviews might be helpful but there is lot of noise in it and you need to find out which dishes people are recommending.
#### This program will read all the restaurant, review information from Zomato and do a basic sentiment analysis telling the top 3 dishes in each restaurant

## Lets begin by installing some libraries we would require

In [223]:
!pip install dropbox

In [224]:
!pip install contractions

In [225]:
!pip install TextBlob

## I will be doing REST API calls to ZOMATO API so importing libraries needed for it

In [226]:
import requests as rq
import json
import pandas as pd
import dropbox
from pandas.io.json import json_normalize

In [227]:
headers = {}

## Lets begin by defining a module for each function

In [228]:
def initialize(api_key):
    '''
    func to intialize the headers for the request URL like api key etc.
    '''
    headers = {
    'Accept': 'application/json',
    'user-key': api_key,
    }
    return headers

In [229]:
def make_query(func,args):

    '''
    func - the call made to the API. eg. reviews/locations/restaurants/collections etc
    args - arguments to be passed alongwith a particular func call
    '''

    #construct the query
    req_url = "https://developers.zomato.com/api/v2.1/"
    req_url=req_url+func
    key_index=0
    for key, value in args.items() :
        if key_index == 0:
            req_url=req_url+"?"+key+"="+value
        else:
            req_url=req_url+"&"+key+"="+value
        key_index+=1
    
    return execute_query(req_url)

In [230]:
def execute_query(query):
    '''
    query - object of class zomatoApiRequest
    headers - dict of meta data for API call
    '''
    response=rq.get(query,headers=headers)
    return(response.json())

In [233]:
def getLocation(location_name):
    '''	
    get details of most relevant locations searched by name
    '''
    func = "locations"
    args = { 'query': location_name }
    output = make_query(func,args)
    if (checkKey(output,'location_suggestions') == 1):
        print("location found")
    else:
        print("location key not found in this iteration")
    return output['location_suggestions'][0]

In [234]:
def getCollections(city_id):
    '''	
    get details of collection
    '''
    func = "collections"
    args =dict()
    args["city_id"] = city_id
    output = make_query(func,args)
    if (checkKey(output,'collections') == 1):
        print("collections found ")
    else:
        print("collections key not found in this iteration")
    return(output['collections'])

In [235]:
def getReviews(res_id):
    '''	
    get details of reviews
    '''
    func = "reviews"
    args =dict()
    args["res_id"] = res_id
    output = make_query(func,args)
    if (checkKey(output,'reviews_shown') == 1):
        print("reviews found")
    else:
        print("reviews key not found in this iteration")
    return(output['user_reviews'])

In [236]:
def search(city_id,collection_id,start=0,count=1):
    '''
    get restaurants for the given location(entity_id); other args - cuisines/collections/category and sorting & counts for fetching how many at a time
    '''
    func = "search"
    args = { 'entity_id':city_id, 'entity_type':'city', 'start':str(start), 'count':str(count),'collection_id':collection_id,'sort':'rating','order':'desc'}
    output = make_query(func,args)
    return(output)

In [237]:
def checkKey(dict, key):
    '''	
    func to check if a dictionary key exists before taking any actions on it
    '''
    if key in dict.keys(): 
        return 1
    else: 
        return 0

In [238]:
def extract_resto_and_reviews(collection_id,output,total_results):
    '''	
    func to extract all restaurants within a collection. This will iterate multiple times as one API call gives on 20 restaurants
    We will populate all the data from the JSON objects into individual lists & finally append to a dictonary which can be easily converted to a pandas frame
    '''
    restro_temp_list = []
    resto_index = 0
    restro_temp_list.append(output['restaurants'])
    #print("Length of resto temp list ",len(restro_temp_list))
    for i in range(0,len(restro_temp_list)): #always len(restro_temp_list) = 1 
        try:
            for rest_dict_val in restro_temp_list[i]: #loop for restaturants within a collection                   

                if resto_index <=total_results: #till we get data for all restaurants in this collection  
                    if (checkKey(rest_dict_val,'restaurant') == 1):
                        #print("processing res_id ", rest_dict_val['restaurant']['id'])
                        collection_id_list.append(collection_id)
                        rest_id_list.append(rest_dict_val['restaurant']['id'])
                        rest_name_list.append(rest_dict_val['restaurant']['name'])
                        rest_locality_list.append(rest_dict_val['restaurant']['location']['locality'])
                        rest_user_rating_list.append(rest_dict_val['restaurant']['user_rating']['aggregate_rating'])

                        rev_output_list = getReviews(str(rest_dict_val['restaurant']['id']))

                        for rev_text_dict_val in rev_output_list: #loop for reviews within a restaurant
                            if (checkKey(rev_text_dict_val,'review') == 1):
                                if rev_text_dict_val['review']['review_text'] != '':
                                    review_rest_id_list.append(rest_dict_val['restaurant']['id'])
                                    review_id_list.append(rev_text_dict_val['review']['id'])
                                    review_text_list.append(rev_text_dict_val['review']['review_text'])
                                    review_rating_list.append(rev_text_dict_val['review']['rating'])                        

                        resto_index = resto_index+1 
        except KeyError:
            pass

In [239]:
def exhaustiveSearch(city_id,collection_id,collection_res_count):
    '''
    The basic call to SEARCH API of ZOMATO
    API allows retrieving only upto 20 results at a time so we interate in same collection till all restaurants obtained
    '''
    start = 0
    cnt = 20
    resto_index = 0
    offset = start
    
    output = search(city_id,collection_id,start=offset,count=cnt)
    total_results = output['results_shown']
    #print("results_shown = ", total_results)
    #print("collection_res_count = ",collection_res_count)
    
    if total_results == collection_res_count: #1st extraction itself gave all restaurants
        print("ONLY 1 CALL ENOUGH to get all restaurants")        
        extract_resto_and_reviews(collection_id,output,total_results)               
    else:
        print("WILL BE DOING MORE CALLS to get all restaurants")
        extract_resto_and_reviews(collection_id,output,total_results) #1st extraction as is since output is already obtained above 
        
        offset = start+total_results #determine new offset for next search
                
        while (offset <=collection_res_count):
            #print("offset=",offset)
            output = search(city_id,collection_id,start=offset,count=cnt)
            
            total_results = output['results_shown']
            print("new results_shown = ", total_results)
            
            extract_resto_and_reviews(collection_id,output,total_results)
            
            offset = offset+total_results #determine new offset for next search
            if(total_results == 0): #no more results coming
                print("no more restaurants data to fetch")
                break
                
    all_review_data = {'rest_id':review_rest_id_list,'review_id':review_id_list, 
                       'review_text':review_text_list,'review_rating':review_rating_list}
    all_resto_data = {'collection_id':collection_id_list,'rest_id':rest_id_list, 
                  'rest_name':rest_name_list,'rest_locality':rest_locality_list,
                  'rest_user_rating':rest_user_rating_list} 

    return(all_resto_data,all_review_data)

## The MAIN code processing begins from here

In [241]:
if __name__ == "__main__":
    api_key = "19b2c1a3c8e2493d77b2231b99696407"
    headers = initialize(api_key)

    user_location=input("Enter location:")
    
    location_result = getLocation(user_location)
    #print(location_result)
    
    collection_list = getCollections(str(location_result['city_id']))

Enter location:Mumbai
location found
collections found 


In [242]:
#collection_list

In [243]:
df_collections = pd.DataFrame.from_dict(json_normalize(collection_list), orient='columns')

In [244]:
df_collections.shape

(48, 7)

In [245]:
new_columns={'collection.collection_id':'collection_id','collection.description':'description','collection.res_count':'res_count','collection.title':'title'}
unwanted_columns=['collection.image_url','collection.share_url','collection.url']
df_collections = df_collections.rename(columns=new_columns)
df_collections = df_collections.drop(columns=unwanted_columns,axis=1)

In [246]:
df_collections.head()

,collection_id,description,res_count,title
0,1,Most popular restaurants in town this week,30,Trending This Week
1,274852,The hunt for the highest-rated restaurants in ...,250,"Great Food, No Bull"
2,29,The best new places in town,32,Newly Opened
3,304361,Binge. Chug. Groove!,15,Mumbai's Best Food & Party Destination - The Orb
4,490,These eateries make healthy food a tasty affair,32,Healthy Living


## We have now collected all the collections & I will only fetch data for first 4 collections in this program

In [247]:
df_sample_collection = df_collections.head(4)

In [248]:
df_sample_collection

,collection_id,description,res_count,title
0,1,Most popular restaurants in town this week,30,Trending This Week
1,274852,The hunt for the highest-rated restaurants in ...,250,"Great Food, No Bull"
2,29,The best new places in town,32,Newly Opened
3,304361,Binge. Chug. Groove!,15,Mumbai's Best Food & Party Destination - The Orb


## Creating dataframe objects to hold review & restaurant data

In [249]:
df_reviews = pd.DataFrame(columns = ['rest_id', 'review_id','review_text','review_rating'])
df_restaurant = pd.DataFrame(columns = ['collection_id','rest_id', 'rest_name','rest_locality','rest_user_rating'])

## For every collection we get all restos & their reviews

In [250]:
for index in df_sample_collection.index: #loop over each collection (4 for now)
    all_review_data = {}
    all_resto_data = {}
    rest_id_list = []
    rest_name_list = []
    rest_locality_list = []
    rest_user_rating_list = []
    collection_id_list = []
    review_rest_id_list = []
    review_id_list = []  
    review_text_list = []
    review_rating_list = []
    
    coll_res_count = df_sample_collection['res_count'][index]
    print("processing collection = ",df_sample_collection['collection_id'][index])
    
    all_resto_data,all_review_data = exhaustiveSearch(str(location_result['city_id']),str(df_sample_collection['collection_id'][index]),coll_res_count)
    df_restaurant = df_restaurant.append(pd.DataFrame(all_resto_data),ignore_index=True) #append all resto data to a frame
    df_reviews = df_reviews.append(pd.DataFrame(all_review_data),ignore_index=True) #append all resto data to a frame
    print("")

print("processed all collections")

processing collection =  1
ONLY 1 CALL ENOUGH to get all restaurants
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found

processing collection =  274852
WILL BE DOING MORE CALLS to get all restaurants
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
new results_shown =  20
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found
reviews found


## We have now collected restaurant and review data in pandas. Optionally you can export this to a dropbox for reviewing independtly on your PC
## Use your DROPBOX API key in below function

In [251]:
def upload_file(file_from, file_to):
    dbx = dropbox.Dropbox("JRedVn3NSbAAAAAAAAAAHILFf6nwfXdAL3zdcH7N1INQXlpsfky6X6CUZ_g7qtuI")
    f = open(file_from, 'rb')
    dbx.files_upload(f.read(), file_to) 

## There is possibility that a resturant might be scanned in mutliple collections for purpose of further analysis lets remove duplicates

In [253]:
df_restaurant = df_restaurant.drop_duplicates(['rest_id'], keep='first')
df_restaurant = df_restaurant.reset_index(drop=True)
df_reviews = df_reviews.drop_duplicates(['review_id'], keep='first')
df_reviews = df_reviews.reset_index(drop=True)

In [254]:
df_restaurant.shape

(159, 5)

In [255]:
df_reviews.shape

(601, 4)

In [256]:
df_restaurant.to_csv('restaurant.csv', header=True)
df_reviews.to_csv('reviews.csv', header=True)

In [257]:
#file1_from = 'restaurant.csv'
#file1_to = '/DataScience/restaurant.csv'
#file2_from = 'reviews.csv'
#file2_to = '/DataScience/reviews.csv'

#upload_file(file1_from,file1_to)
#upload_file(file2_from,file2_to)

In [258]:
#df_restaurant['rest_id'].value_counts()

In [259]:
#df_reviews['review_id'].value_counts()

## Now I am importing libraries needed for NLP like NLTK, Text Blob, Spacy

In [260]:
import nltk
import re
import unicodedata
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from textblob import TextBlob

In [261]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dsxuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /home/dsxuser/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [262]:
stopword = stopwords.words('english')
to_remove = ['and', 'the']
new_stopwords = list(set(stopword).difference(to_remove))

## Defining function for each task in NLP pre-processing

In [263]:
def to_lowercase(text):
    return text.lower()

In [264]:
def remove_html(text):
    return (re.sub('<[^<]+?>','', text))

In [265]:
def remove_non_ascii(text):
    return (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

In [266]:
def remove_punctuations(text):
    return (re.sub('[~!@#$%^&?,:";-]', '', text))

In [267]:
def remove_numbers(text):
    return (''.join(c for c in text if not c.isdigit()))

In [268]:
def remove_stopwords(text):
    word_tokens = nltk.word_tokenize(text)
    return(' '.join([word for word in word_tokens if word not in new_stopwords]))

In [166]:
def lemmatize(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(text)
    return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in word_tokens])

In [269]:
def remove_contractions(text):
    '''func to convert contractions to full words e.g. can't will become cannot'''
    return ' '.join([contractions.fix(word) for word in text.split()])

In [270]:
def do_spell_corrections(text_blob):
    return(text_blob.correct())

In [271]:
def get_ngram_noun_phrases(sentence, ngrams = 2):
    '''
    func to get all noun phrases using Text Blob. A dish name might be all nouns but could be a 2 letter word or 3 or 4 or 5
    this will extract all noun phrases from the review's blob objects and put in a list
    '''
    sent_blob = TextBlob(str(sentence))
    ngram_list = sent_blob.ngrams(n=ngrams)   
    #print(ngram_list)
    #print("")
    for pair in ngram_list:
        ngram = ' '.join(pair)
        word_blob = TextBlob(ngram)
        for np in word_blob.noun_phrases:
            #print(np)
            if np not in noun_phrases_list:
                noun_phrases_list.append(np)

In [272]:
def clean_review(text,is_Lower=True,remove_html=True,remove_nascii=True,remove_punct=True,remove_cont=True,remove_stop=True,is_lemma=False):
    '''
    func to pre-process the review text. You can toggle a param if you dont want particular pre-processing task to happen
    '''
    if is_Lower:
        text = to_lowercase(text)
    elif remove_html:
        text= remove_html(text)
    elif remove_nascii:
        text= remove_non_ascii(text)
    elif remove_punct:
        text= remove_punctuations(text)
    elif remove_cont:
        text= remove_contractions(text)
    elif remove_stop:
        text= remove_stopwords(text)
    elif is_lemm:
        text= lemmatize(text)
    return (text)    

In [273]:
def sentiment_textblob(feedback): 
    '''
    func to map the sentence polarity to a user defined sentiment label
    '''
    senti = TextBlob(feedback) 
    polarity = senti.sentiment.polarity 
    if -1 <= polarity < -0.5: 
        label = 'very bad' 
    elif -0.5 <= polarity < -0.1: 
        label = 'bad' 
    elif -0.1 <= polarity < 0.2: 
        label = 'ok' 
    elif 0.2 <= polarity < 0.6: 
        label = 'good' 
    elif 0.6 <= polarity <= 1: 
        label = 'best' 
    
    return (polarity, label) 

In [274]:
def generate_noun_pharses(rev_blob):
    '''
    func to generate all bnoun phrases from a blob object
    '''
    for sent in rev_blob.sentences:   
        get_ngram_noun_phrases(sent,2) #bigram phrases
        get_ngram_noun_phrases(sent,3) #trigram phrases
        get_ngram_noun_phrases(sent,4) 
        get_ngram_noun_phrases(sent,5)

In [275]:
def generate_possible_menu_list(noun_phrases_list):
    '''
    func that will create a list where the noun phrase contains all nouns which means its a possible dish name
    '''
    for element in noun_phrases_list:
        tag_list = []
        phrase_blob = TextBlob(element)
        tag_list = phrase_blob.tags
        #print("tag_list ",tag_list)
        if ((tag_list[0][1] == 'NN' or tag_list[0][1] == 'NNS') and (tag_list[-1][1] == 'NN' or tag_list[-1][1] == 'NNS')):
            possible_menu_list.append(element)

In [276]:
def remove_duplicate_noun_phrases(possible_menu_list):
    '''
    We could get similar dish names while parsing each noun phrase
    e.g. chicken noodles vs spicy chicken noodles. In such cases will keep only the phrase with longest length
    '''
    for i, elements in enumerate(possible_menu_list):
        try:
            thiselem = str(elements)
            matching_elements = [s for s in possible_menu_list if thiselem in s]
            #print(matching_elements)
            for j, val in enumerate(matching_elements):
                curr_val = str(val)
                next_val = str(matching_elements[(j + 1) % len(matching_elements)])

                if len(curr_val) > len(next_val):
                    #print("Removing ",next_val)
                    if next_val not in exclude_list:
                        exclude_list.append(next_val)
                elif len(curr_val) < len(next_val):
                    #print("Removing ",curr_val)
                    if curr_val not in exclude_list:
                        exclude_list.append(curr_val)
        except:
            pass
    
    for x in exclude_list:
        try:
            possible_menu_list.remove(x)
        except:
            pass

In [277]:
def analyze_review_to_get_menu_and_sentiment(rev_id,text):
    '''
    func to analyze each review text
    1. Convert to a BLOB
    2. Optional spell correct. Turned this off because if spelling changes then searching in actual text becomes difficult
    3. Get noun phrases
    4. Get possible menu list items (all nouns POS)
    5. Retain menu items with max length
    6. Search each item with each sentence of the given review text and if found
       a. Get sentiment score, label of that sentence
       b. break
       c. Store in list for adding to a pandas frame
    '''
    
    cl_review = clean_review(text,True,True,True,True,True,True,False)
    cl_review_s = clean_review(text,True,True,True,True,True,False,False)
    
    rev_blob = TextBlob(cl_review)
    rev_blob_s = TextBlob(cl_review_s)
    
    #rev_blob = do_spell_corrections(rev_blob)
    #rev_blob_s = do_spell_corrections(rev_blob_s)
    
    generate_noun_pharses(rev_blob)
    #print("noun_phrases_list = ", noun_phrases_list)
    
    generate_possible_menu_list(noun_phrases_list)
    #print("possible menu_list = ",possible_menu_list) 
    
    remove_duplicate_noun_phrases(possible_menu_list)
    #print("possible menu_list after duplicates removal=",possible_menu_list)
    
    raw_sentence_list = sent_tokenize(str(rev_blob_s))
    
    detected_sentence = ''
    polarity= -99 
    label = ''
    for m in possible_menu_list:
        for sentence in raw_sentence_list:
            if m in sentence:
                (polarity, label) = sentiment_textblob(str(sentence))
                
                detected_sentence = sentence
                break
        rev_id_list.append(rev_id)
        menu_item_list.append(m)
        detected_sentence_list.append(detected_sentence)
        sent_polarity_list.append(polarity)
        sentiment_label_list.append(label)

In [278]:
#df_reviews = df_reviews.drop(columns="clean_review_text",axis=1)

## Using the clean function to create a column for cleaned review for reference

In [281]:
df_reviews['clean_review_text'] = df_reviews.review_text.apply(clean_review)

In [282]:
df_reviews.head()

,rest_id,review_id,review_text,review_rating,clean_review_text
0,18234205,46259136,"One Street Bandra, the elder sister of Bastian...",4,"one street bandra, the elder sister of bastian..."
1,18234205,45648284,"Ashish was really good at serving us, his reco...",4,"ashish was really good at serving us, his reco..."
2,18365166,46457602,"Went there in a group of 10 on Friday night, t...",4,"went there in a group of 10 on friday night, t..."
3,18365166,41084301,Hideout cafe.. it's really hidden place.... Bu...,5,hideout cafe.. it's really hidden place.... bu...
4,18365166,40358430,Hideout café hides in plain sight at the High ...,4,hideout café hides in plain sight at the high ...


## Processing all reviews from reviews frame and store in new frame

In [283]:
df_analyzed_reviews = pd.DataFrame(columns = ['review_id','menu_item','sentiment_score','sentiment_label','what_people_said'])
for index in df_reviews.index: #loop over each review
    #print("processing review = ",df_reviews['review_id'][index])
    
    rev_id_list = []
    menu_item_list = []
    detected_sentence_list = []
    sent_polarity_list = []
    sentiment_label_list = []
    
    noun_phrases_list = []
    possible_menu_list = []
    exclude_list = []
    raw_sentence_list=[]
    
    analyze_review_to_get_menu_and_sentiment(df_reviews['review_id'][index],str(df_reviews['review_text'][index]))
    
    #print(menu_item_list)
    #print(sent_polarity_list)
    #print(sentiment_label_list)
    #print(detected_sentence_list)
    #break
    
    all_cleaned_review_data = {'review_id':rev_id_list,
                           'menu_item':menu_item_list,'sentiment_score':sent_polarity_list,
                           'sentiment_label':sentiment_label_list,'what_people_said':detected_sentence_list}

    df_analyzed_reviews = df_analyzed_reviews.append(pd.DataFrame(all_cleaned_review_data),ignore_index=True) #append all analyzed review data to a frame

print("processed all reviews")

processed all reviews


## Lets sort each menu item within a review in descending order of score

In [284]:
df_analyzed_reviews = df_analyzed_reviews.sort_values(['review_id','sentiment_score'],ascending=[1, 0])

In [285]:
df_analyzed_reviews.head()

,review_id,menu_item,sentiment_score,sentiment_label,what_people_said
57,40174927,name suits,0.000000,ok,the name suits this cafe it's like you have to...
58,40174927,beer pizza,0.000000,ok,the name suits this cafe it's like you have to...
56,40323829,hukah place,0.500000,good,the ambiance was more of a hukah place...
55,40323829,pink sauce spaghetti,0.334375,good,this cafe was earlier famous for hukah... now ...
54,40358430,wine sangria,0.500000,good,red wine sangria : again surprisingly awesome ...


## Now we will be merging all frames using pandas join method to arrive at final output
## Final output looks like
### COLLECTION DETAILS | RESTAURANT DETAILS | MENU ITEMS | SENTI SCORE of what people said about menu 

In [287]:
df_restaurant[['collection_id']] = df_restaurant[['collection_id']].apply(pd.to_numeric) 

In [288]:
df_temp1 = pd.merge(df_collections,df_restaurant,on='collection_id',how='inner') #merged collections with resto frame

In [289]:
df_temp2 = pd.merge(df_temp1,df_reviews,on='rest_id',how='inner') #merged with review frame

In [290]:
df_final = pd.merge(df_temp2,df_analyzed_reviews,on='review_id',how='inner') #merged with analyzed reviews

In [291]:
unwanted_columns=['collection_id','res_count','rest_id','review_id','review_text']
df_final = df_final.drop(columns=unwanted_columns,axis=1)

In [292]:
df_final.head()

,description,title,rest_name,rest_locality,rest_user_rating,review_rating,clean_review_text,menu_item,sentiment_score,sentiment_label,what_people_said
0,Most popular restaurants in town this week,Trending This Week,One Street,"Linking Road, Bandra West",4.1,4,"one street bandra, the elder sister of bastian...",cookie stands,1.000000,best,the cookie stands out for its taste and goes p...
1,Most popular restaurants in town this week,Trending This Week,One Street,"Linking Road, Bandra West",4.1,4,"one street bandra, the elder sister of bastian...",chocolate sauce,1.000000,best,the cookie stands out for its taste and goes p...
2,Most popular restaurants in town this week,Trending This Week,One Street,"Linking Road, Bandra West",4.1,4,"one street bandra, the elder sister of bastian...",comfort finger food,0.533333,good,"for comfort finger food and good drinks, it of..."
3,Most popular restaurants in town this week,Trending This Week,One Street,"Linking Road, Bandra West",4.1,4,"one street bandra, the elder sister of bastian...",’ s,0.533333,good,"for comfort finger food and good drinks, it of..."
4,Most popular restaurants in town this week,Trending This Week,One Street,"Linking Road, Bandra West",4.1,4,"one street bandra, the elder sister of bastian...",multiple options,0.525000,good,"the service is good, recommendations are excel..."
